In [53]:
// Set log level to ERROR (less verbose)
sc.setLogLevel("ERROR")
// Import libs

import  org.apache.spark._ 
import  spark.implicits._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.SQLContext
import org.apache.spark.{SparkContext, SparkConf}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.Imputer
import org.apache.spark.sql._
import SQLContext._

import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row
import scala.util.hashing.MurmurHash3
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator  
import org.apache.spark.mllib.evaluation.MulticlassMetrics  
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics  
import org.apache.spark.ml.classification.LogisticRegression  

import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit, CrossValidator}  
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, OneHotEncoderEstimator,VectorIndexer}  
import org.apache.spark.ml.linalg.Vectors  
import org.apache.spark.ml.{Pipeline,PipelineModel}  
import org.apache.log4j._  

import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.feature.{IndexToString, StringIndexer}
import org.apache.spark.sql.DataFrame
Logger.getLogger("org").setLevel(Level.ERROR)  
import java.io.{File, PrintWriter}


import org.apache.spark._
import spark.implicits._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.SQLContext
import org.apache.spark.{SparkContext, SparkConf}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.Imputer
import org.apache.spark.sql._
import SQLContext._
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row
import scala.util.hashing.MurmurHash3
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mll...

In [7]:
val weather_RFI_df = spark.
        read.
        option("inferSchema", "true").
        option("header","true").
        option("mode","DROPMALFORMED").
        option("delimiter", ",").
        csv("weatherAUS-clean-FINAL.csv").
        na.drop()

weather_RFI_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 31 more fields]


In [8]:
weather_RFI_df.count()

res2: Long = 78309


In [9]:
weather_RFI_df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: double (nullable = true)
 |-- MaxTemp: double (nullable = true)
 |-- Rainfall: double (nullable = true)
 |-- Evaporation: double (nullable = true)
 |-- Sunshine: double (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: double (nullable = true)
 |-- WindSpeed3pm: double (nullable = true)
 |-- Humidity9am: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Cloud9am: double (nullable = true)
 |-- Cloud3pm: double (nullable = true)
 |-- Temp9am: double (nullable = true)
 |-- Temp3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- RainTomorrow: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- RainToday_Sc: integer (nullable = true)
 |--

In [ ]:
//// explnation

In [32]:
val weather_RFI_df_s = weather_RFI_df.selectExpr("Date","Location",
                       
                        "MaxTemp",
                       
                          "Humidity3pm", 
                        
                         "Pressure3pm",
                         
                        "Temp3pm",
                        "Temp9am",
                         "RainToday_Sc",
                         "cast (Month as String) Month",
                         "RainTomorrow_Sc"
                      
                                                  )

weather_RFI_df_s: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 8 more fields]


# Explanation about the approach on why we are considering only location based data


In [34]:
val weather_SYD_df=weather_RFI_df_s.filter($"Location"==="Sydney")toDF()

weather_SYD_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 8 more fields]


In [35]:
weather_RFI_df.count()
weather_SYD_df.count()

res13: Long = 3337


# Setting the label column(Rain tomorrow)

In [36]:
val weather_SYD_df_lb = (weather_SYD_df.select(weather_SYD_df("RainTomorrow_Sc").as("label"), 
           $"Location",
           $"Date",
        
          $"MaxTemp", 
         $"Month",
         $"RainToday_Sc",
        
          $"Humidity3pm",
          
          $"Pressure3pm", 
           $"Temp9am", 
          $"Temp3pm"))

weather_SYD_df_lb: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 8 more fields]


# Indexing the month column(categorical)-To capture the seasonal pattern

In [37]:
val Indexer=new StringIndexer().setInputCol("Month").setOutputCol("MonIn")

Indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_982802b57382


# Encoding the MonIn( column)

In [38]:
val Encoder=new OneHotEncoderEstimator().setInputCols(Array("MonIn")).setOutputCols(Array("MonEn"))

Encoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_dcc4d15f8c0e


# Setting the assembler

In [40]:
val assembler=(new VectorAssembler().setInputCols(Array(
                                                        "RainToday_Sc",
                                                         "MaxTemp",
                                                        "Humidity3pm",
                                                         "Temp3pm",
                                                         "MonEn",
                                                      
                                                        "Pressure3pm"
                                                        )).setOutputCol("features"))

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_d455f33b9cf3


# Splitting the data into test and train sets

In [41]:
val Array(training,test)=weather_SYD_df_lb.randomSplit(Array(0.7,0.3),seed=999)

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Location: string ... 8 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Location: string ... 8 more fields]


# Random Forrest 

In [43]:
  val rf = new RandomForestClassifier()
      .setLabelCol("label")
      .setFeaturesCol("features")
      .setNumTrees(10)
      .setMaxBins(41)
rf

rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_7a1b9d928105
res15: org.apache.spark.ml.classification.RandomForestClassifier = rfc_7a1b9d928105


# Setting the pipeline

In [44]:
val M_pipeline=new Pipeline().setStages(Array(Indexer,Encoder,assembler,rf))

M_pipeline: org.apache.spark.ml.Pipeline = pipeline_1951312b7192


In [45]:
val Trg_model=M_pipeline.fit(training)

Trg_model: org.apache.spark.ml.PipelineModel = pipeline_1951312b7192


In [47]:
Trg_model.write
  .overwrite()
  .save("spark-RandomForrest-model")

In [48]:
// And load it back in during production
val RegModel = PipelineModel.load("spark-RandomForrest-model")

RegModel: org.apache.spark.ml.PipelineModel = pipeline_1951312b7192


In [49]:
val results =RegModel.transform(test)

results: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 14 more fields]


In [50]:
  val evaluator = new MulticlassClassificationEvaluator()
      .setLabelCol("label")
      .setPredictionCol("prediction")
      .setMetricName("accuracy")

evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_6c9a1c17129d


In [51]:
val accuracy = evaluator.evaluate(results)

accuracy: Double = 0.8394088669950739


# Trying with full dataset

In [ ]:
weather_RFI_df_s

In [59]:
val weather_full_df_lb = (weather_RFI_df_s.select(weather_SYD_df("RainTomorrow_Sc").as("label"), 
           $"Location",
           $"Date",
        
          $"MaxTemp", 
         $"Month",
         $"RainToday_Sc",
        
          $"Humidity3pm",
          
          $"Pressure3pm", 
           $"Temp9am", 
          $"Temp3pm"))

weather_full_df_lb: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 8 more fields]


# Indexing the month column(categorical)-To capture the seasonal pattern

In [60]:
val Indexer=new StringIndexer().setInputCol("Month").setOutputCol("MonIn")
val Encoder=new OneHotEncoderEstimator().setInputCols(Array("MonIn")).setOutputCols(Array("MonEn"))
val assembler=(new VectorAssembler().setInputCols(Array(
                                                        "RainToday_Sc",
                                                         "MaxTemp",
                                                        "Humidity3pm",
                                                         "Temp3pm",
                                                         "MonEn",
                                                      
                                                        "Pressure3pm"
                                                        )).setOutputCol("features"))

Indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_a2d4760f7912
Encoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_330c31f00b09
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_5d8cedddb62e


In [61]:
weather_full_df_lb.count()

res22: Long = 78309


In [65]:
// Split the data
val Array(training_full,test_full)=weather_full_df_lb.randomSplit(Array(0.7,0.3),seed=999)

training_full: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Location: string ... 8 more fields]
test_full: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Location: string ... 8 more fields]


In [66]:
val rf_full = new RandomForestClassifier()
      .setLabelCol("label")
      .setFeaturesCol("features")
      .setNumTrees(10)
      .setMaxBins(41)
rf_full

rf_full: org.apache.spark.ml.classification.RandomForestClassifier = rfc_0cc9ca9f683a
res24: org.apache.spark.ml.classification.RandomForestClassifier = rfc_0cc9ca9f683a


In [67]:
// create the pipe line 

val M_pipeline_full =new Pipeline().setStages(Array(Indexer,Encoder,assembler,rf_full))

M_pipeline_full: org.apache.spark.ml.Pipeline = pipeline_27876b697add


In [69]:
val Trg_model_full=M_pipeline.fit(training_full)

Trg_model_full: org.apache.spark.ml.PipelineModel = pipeline_1951312b7192


In [70]:
Trg_model_full.write
  .overwrite()
  .save("spark-RandomForrest-model-full")

In [71]:
val RegModel_full = PipelineModel.load("spark-RandomForrest-model-full")

RegModel_full: org.apache.spark.ml.PipelineModel = pipeline_1951312b7192


In [72]:
val results_full =RegModel_full.transform(test_full)

results_full: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 14 more fields]


In [73]:
  val evaluator_full = new MulticlassClassificationEvaluator()
      .setLabelCol("label")
      .setPredictionCol("prediction")
      .setMetricName("accuracy")

evaluator_full: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_dc2a33402004


In [74]:
val accuracy_full = evaluator_full.evaluate(results)

accuracy_full: Double = 0.8394088669950739
